## Plot  Discovery Significance vs Mass

This notebook makes a p0 plot a scan of the discovery significance vs mass for the $H \to \gamma \gamma$ example.

We are going to fit the observed spectrum for different fixed value of the mass $M$ and keeping also the width of the signal constant.

We compute the significance using the asymptotic approximation

In [ ]:
%jsroot on

#### Set the number of expected events. This is used to compute the expected significance

In [ ]:
int nevt_expected = 250; 

#### Reading input file 

We read first the file with the workspace and we retrieve the needed objects

In [ ]:
// Check if example input file exists
TFile *file = TFile::Open("HiggsBinModel.root");

// get the workspace out of the file
RooWorkspace* w = (RooWorkspace*) file->Get("w");

// get the data out of the file
RooAbsData* data = w->data("data");

// get the modelConfig (S+B) out of the file
// and create the B model from the S+B model
RooStats::ModelConfig*  sbModel = (RooStats::ModelConfig*) w->obj("ModelConfig");
sbModel->SetName("S+B Model");      
RooRealVar* poi = (RooRealVar*) sbModel->GetParametersOfInterest()->first();
// the workspace contains the number of observed events
int nevt_obs = poi->getVal(); 
poi->setVal(nevt_expected);  // set POI snapshot in S+B model for expected significance
sbModel->SetSnapshot(*poi);
RooStats::ModelConfig * bModel = (RooStats::ModelConfig*) sbModel->Clone();
bModel->SetName("B Model");      
poi->setVal(0);
bModel->SetSnapshot( *poi  );

#### Plot of fit result

Before scanning the mass values for the significance, we plot the best fit result 

In [ ]:
massPlot = w->var("x")->frame();
data->plotOn(massPlot);
pdf = sbModel->GetPdf();
// set in pdf the correct number of observed events from the fit 
poi->setVal(nevt_obs);
pdf->plotOn(massPlot);
pdf->plotOn(massPlot, RooFit::Components("bmodel"),RooFit::LineStyle(kDashed));
pdf->plotOn(massPlot, RooFit::Components("smodel"),RooFit::LineColor(kRed));
pdf->paramOn(massPlot);

massPlot->Draw();
gPad->Draw();

In [ ]:
vector<double> masses;
vector<double> p0values;
vector<double> p0valuesExpected;
double massMin, massMax; 

###   Loop on the mass values

Perform a loop between the given values and perform an asymptotic hypothesis test to compute the p-value at each scanned mass point

In [ ]:
massMin = 112;
massMax = 158;


int npoints = 30;
for( double mass=massMin; mass<=massMax; mass += (massMax-massMin)/double(npoints) )
  {
      
      
     cout << endl << endl << "Running for mass: " << mass << endl << endl;
     w->var("mass")->setVal( mass );

     RooStats::AsymptoticCalculator ac(*data, *sbModel, *bModel);
     ac.SetOneSidedDiscovery(true);  // for one-side discovery test                                      
     RooStats::AsymptoticCalculator::SetPrintLevel(-1);


     RooStats::HypoTestResult* asymCalcResult = ac.GetHypoTest();
 
     asymCalcResult->Print();
     
     masses.push_back( mass );
     p0values.push_back( asymCalcResult->NullPValue() );
  
     double expectedP0 = RooStats::AsymptoticCalculator::GetExpectedPValues(  asymCalcResult->NullPValue(),  asymCalcResult->AlternatePValue(), 0, false);
     p0valuesExpected.push_back( expectedP0 );
     std::cout << "expected p0 = " << expectedP0 << std::endl;

  }


Create a plot with the obtained p-values

In [ ]:
  TGraph * graph1  = new TGraph(masses.size(),&masses[0],&p0values[0]);
  TGraph * graph2  = new TGraph(masses.size(),&masses[0],&p0valuesExpected[0]);

  graph1->SetMarkerStyle(20);
  graph1->Draw("APC");
  graph2->SetLineStyle(2);
  graph2->Draw("C");
  graph1->GetXaxis()->SetTitle("mass");
  graph1->GetYaxis()->SetTitle("p0 value");
  graph1->SetTitle("Significance vs Mass");
  graph1->SetMinimum(graph2->GetMinimum());
  graph1->SetLineColor(kBlue);
  graph2->SetLineColor(kRed);

  // draw the signma lines
  double xmin = graph1->GetXaxis()->GetXmin();
  double xmax = graph1->GetXaxis()->GetXmax();
  for (int sigma =1; sigma < 5; sigma++)  {
      double p =  ROOT::Math::normal_cdf_c(sigma,1);
      auto line = new TLine(xmin, p, xmax, p);
      line->Draw();
      auto stext = new TText(xmax,p,TString::Format("%d-sigma",sigma));
      stext->Draw();
  }

  gPad->SetLogy(true);
  graph1->SetMinimum(1.E-5);

In [ ]:
gPad->Draw();